In [17]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f
!pip install selenium beautifulsoup4 webdriver_manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from collections import defaultdict
import datetime
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

--2025-10-22 18:13:31--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 192.178.156.93, 192.178.156.136, 192.178.156.190, ...
Connecting to dl.google.com (dl.google.com)|192.178.156.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120234260 (115M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 114.66M   393MB/s    in 0.3s    

2025-10-22 18:13:32 (393 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [120234260/120234260]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (141.0.7390.122-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Packa

In [18]:
# 1️ Configure Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # run in background (no GUI)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox") # Add this line for Colab environment
chrome_options.add_argument("--disable-dev-shm-usage") # Add this line for Colab environment
chrome_options.binary_location = '/usr/bin/google-chrome' # Explicitly set the path to the Chrome binary

In [21]:
# Selenium setup (example for Chrome; specify path to chromedriver)
service = Service(ChromeDriverManager().install()) # Use webdriver_manager to install chromedriver
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    # Open the BBC Innovation / Technology section page
    url = 'https://www.bbc.com/innovation/technology'
    driver.get(url)

    # Wait for JS content to load (may need longer wait)
    time.sleep(5)

    # Get the HTML after page load
    html = driver.page_source

    # Use BeautifulSoup to parse the HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Find article elements — adjust selectors to match actual page structure
    articles = soup.select('a.gs‑c‑media__link')  # example selector — may not match
    print(f"Found {len(articles)} articles")

    # Dictionary: month → list of (title, date)
    by_month = defaultdict(list)

    for a in articles:
        title = a.get_text(strip=True)
        # Try to find the publication date nearby
        # For example: <time datetime="2025‑10‑22">22 October 2025</time>
        time_tag = a.find('time')
        if time_tag and time_tag.has_attr('datetime'):
            dt = datetime.datetime.fromisoformat(time_tag['datetime'])
        else:
            # skip if no date is found
            continue

        month_name = dt.strftime('%B')  # e.g., 'October'
        by_month[month_name].append((title, dt.strftime('%Y‑%m‑d')))

    # Print results
    for month in sorted(by_month.keys(), key=lambda m: datetime.datetime.strptime(m, '%B').month):
        print(f"=== {month} ===")
        for title, date in by_month[month]:
            print(f"{date} — {title}")
        print()
finally:
    driver.quit()

Found 0 articles
